
https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|offset%3D50&format=json

Better query:
https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|limit%3D2000&format=json

Much better query: https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|?DBLP%20Series%20ID|?Title|?Acronym|?DOI|limit%3D3000&format=json

# Downloading data

In [7]:
import requests
import json

In [2]:
r = requests.get('https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|?DBLP%20Series%20ID|?Title|?Acronym|?DOI|limit%3D3000&format=json')

In [3]:
r.status_code

200

In [4]:
result = r.json()

In [5]:
result.keys()

dict_keys(['query'])

In [8]:
with open('ConfIDent.json','w') as fw:
    json.dump(result, fw)

# Creating Index

In [10]:
with open('ConfIDent.json','r') as fr:
    confident = json.load(fr)

In [20]:
c_results = confident["query"]["results"]
unique_confs = {}
for k, c_series in c_results.items():
    unique_confs[c_series["displaytitle"]] = k
    if len(c_series["printouts"]["Title"]) > 0:
        unique_confs[c_series["printouts"]["Title"][0]] = k
len(unique_confs)

3705

In [21]:
dblp_confs = {}
for k, c_series in c_results.items():
    if len(c_series["printouts"]["DBLP Series ID"]) > 0:
        dblp_confs[c_series["printouts"]["DBLP Series ID"][0]] = k
len(dblp_confs)

566

In [24]:
# dblp_confs["semweb"]

In [16]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = list(unique_confs.keys())

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

vector_size = embeddings.shape[1]

/opt/anaconda3/envs/dev38/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


(3705, 384)


In [17]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(vector_size)   # build the index
print(index.is_trained)

True


In [18]:
index.add(embeddings)                  # add vectors to the index
print(index.ntotal)

3705


In [25]:
to_save = dict()
to_save["sentences"] = sentences
to_save["index"] = index
to_save["confs"] = unique_confs
to_save["conf_series"] = c_results
to_save["dblp_confs"] = dblp_confs
import pickle
with open('ConfIDent.pickle', 'wb') as handle:
    pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)